# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.


### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.groupby(by = 'race').call.count()

race
b    2435
w    2435
Name: call, dtype: int64

Same number of black and white CVs. 2435 in each group

In [6]:
# create data for black and white and calcualte numver of call backs
black = data[data.race == 'b']
black_callback = black.call.sum()
white = data[data.race == 'w']
white_callback = white.call.sum()
print 'Total black cv %d. Total call backs %d '  % (len(black),black_callback)
print 'Total black cv %d. Total call backs %d '  % (len(white),white_callback)

Total black cv 2435. Total call backs 157 
Total black cv 2435. Total call backs 235 


For black CVs the call back is lower : 157 vs 235 for white

We test the hypothesis:

- Null: H0 = there is no difference between callback rates on resumes from blacks vs. whites
- Alternative: HA = there is a difference between callback rates

In [7]:
stats.ttest_ind(black.call, white.call, equal_var=True)

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9408021031288859e-05)

In [8]:
stats.ttest_ind(black.call, white.call, equal_var=False)

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9429415136459352e-05)

Conclusion: There is a significant difference in callback rates, reject null

In [9]:
# probability of being called back by race
p_black = black_callback/len(black)
p_white = white_callback/len(white)
print "Probability of being called back: \n black: %f ; white %f" %(p_black,p_white)

Probability of being called back: 
 black: 0.064476 ; white 0.096509


In [10]:
# Confidence Interval at 95%
confidence_level = 0.95
z = stats.norm.ppf(confidence_level)
print "Using confidence level %f : z = %f" % (confidence_level,z)
#error margin
error_margin = z * math.sqrt(p_white*(1 - p_white)/len(white) + p_black * (1 - p_black)/len(black))
print 'Error margin: ' ,  error_margin

Using confidence level 0.950000 : z = 1.644854
Error margin:  0.0128025053394


In [11]:
# Confidence interval
print "Confidence interval at %.2f" %confidence_level
[p_white - p_black - z * error_margin, p_white -p_black + z * error_margin]

Confidence interval at 0.95


[0.010974606867863518, 0.053091101551027656]

In [12]:
# Test proportion and find p-value
res = proportions_ztest(np.array([white_callback,black_callback]),np.array([len(white),len(black)]),value=0)
res

(4.1084121524343464, 3.9838868375850767e-05)

In [13]:
print 'P- value = %f' %res[1]

P- value = 0.000040


We reject the Null hypothesis , becauase p-value is small
So we find that there is indeed the race discrimination in the call-backs to the interviews